In [1]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Размеры изображения
img_width, img_height = 256, 256
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 20
# Размер мини-выборки
batch_size = 16
# Количество изображений для обучения
nb_train_samples = 1178
# Количество изображений для проверки
nb_validation_samples = 322
# Количество изображений для тестирования
nb_test_samples = 322

Слой свертки, размер ядра 3х3, количество карт признаков - 32 шт., функция активации ReLU.
Слой подвыборки, выбор максимального значения из квадрата 2х2
Слой свертки, размер ядра 3х3, количество карт признаков - 32 шт., функция активации ReLU.
Слой подвыборки, выбор максимального значения из квадрата 2х2
Слой свертки, размер ядра 3х3, количество карт признаков - 64 шт., функция активации ReLU.
Слой подвыборки, выбор максимального значения из квадрата 2х2
Слой преобразования из двумерного в одномерное представление
Полносвязный слой, 64 нейрона, функция активации ReLU.
Слой Dropout.
Выходной слой, 1 нейрон, функция активации sigmoid

Слои с 1 по 6 используются для выделения важных признаков в изображении, а слои с 7 по 10 - для классификации.

In [14]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [15]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [16]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [17]:
train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Hotdog_Not_Hotdog/train',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 2356 images belonging to 2 classes.


In [18]:
val_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Hotdog_Not_Hotdog/val',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 644 images belonging to 2 classes.


In [19]:
test_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Hotdog_Not_Hotdog/test',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 644 images belonging to 2 classes.


Обучаем модель с использованием генераторов

train_generator - генератор данных для обучения

validation_data - генератор данных для проверки

In [20]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/20
73/73 [==============================] - 9s 100ms/step - loss: 0.6915 - accuracy: 0.5745 - val_loss: 0.7048 - val_accuracy: 0.5344
Epoch 2/20
73/73 [==============================] - 6s 84ms/step - loss: 0.6323 - accuracy: 0.6747 - val_loss: 0.7149 - val_accuracy: 0.5097
Epoch 3/20
73/73 [==============================] - 7s 96ms/step - loss: 0.5945 - accuracy: 0.6903 - val_loss: 0.7163 - val_accuracy: 0.5719
Epoch 4/20
73/73 [==============================] - 6s 81ms/step - loss: 0.5677 - accuracy: 0.7183 - val_loss: 0.7040 - val_accuracy: 0.5487
Epoch 5/20
73/73 [==============================] - 7s 93ms/step - loss: 0.5567 - accuracy: 0.7163 - val_loss: 0.6899 - val_accuracy: 0.6125
Epoch 6/20
73/73 [==============================] - 6s 80ms/step - loss: 0.5532 - accuracy: 0.7260 - val_loss: 0.6824 - val_accuracy: 0.5747
Epoch 7/20
73/73 [==============================] - 7s 92ms/step - loss: 0.5377 - accuracy: 0.7292 - val_loss: 0.6551 - val_accuracy: 0.6156
Epoch 8/20
7

Оцениваем качество работы сети с помощью генератора

In [21]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

In [22]:
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

Аккуратность на тестовых данных: 70.00%
